In [1]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import tensorflow as tf
import tensorflow_hub as hub

physical_devices = tf.config.list_physical_devices('GPU')
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2024-02-25 14:11:48.216451: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-25 14:11:48.216539: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-25 14:11:48.257398: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-25 14:11:48.351337: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-25 14:11:49.345819: W tensorflow/compiler/tf2

In [2]:
df = pd.read_csv("../tags_processed_stages/dafre_tags.csv")
num_classes = df['tags_cat4'].nunique()
print(num_classes)

3486


In [3]:
len(df)

463437

In [4]:
class_counts = df['tags_cat4'].value_counts()
print(class_counts)

def contains_multiple_tags(label, separator=','):
    return separator in label
filtered_df = df[~df['tags_cat4'].apply(contains_multiple_tags, separator=',')]

class_counts = filtered_df['tags_cat4'].value_counts()
print(class_counts)

train_df, temp_test_df = train_test_split(filtered_df, test_size=0.2, stratify=filtered_df['tags_cat4'])
val_df, test_df = train_test_split(temp_test_df, test_size=0.5, stratify=temp_test_df['tags_cat4'])


print(train_df['tags_cat4'].nunique())
print(val_df['tags_cat4'].nunique())
print(test_df['tags_cat4'].nunique())

['hatsune_miku']                                                                                10644
['hakurei_reimu']                                                                                6690
['rumia']                                                                                        3803
['kochiya_sanae']                                                                                3472
['cirno']                                                                                        3470
                                                                                                ...  
['kirisame_marisa', 'rei_(cookie)']                                                                 1
['cecilia_alcott', 'charlotte_dunois', 'huang_lingyin', 'orimura_ichika', 'shinonono_houki']        1
['female_gunner_(dungeon_and_fighter)', 'rose_(elsword)']                                           1
['oosanshouuo-san', 'ranka_lee']                                                  

In [5]:
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [6]:
train_datagen = ImageDataGenerator(rescale=1./255)

In [46]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory='../fullMin256/', 
    x_col='dir', 
    y_col='tags_cat4', 
    target_size=(224, 224), 
    batch_size=32,
    class_mode='categorical',
    color_mode = 'rgb'
)

validation_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory='../fullMin256/',
    x_col='dir',
    y_col='tags_cat4',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=True,
    color_mode = 'rgb'
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory='../fullMin256/',
    x_col='dir',
    y_col='tags_cat4',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    color_mode = 'rgb'
)
x,y  = next(train_generator)
print(x.shape)

Found 369126 validated image filenames belonging to 3260 classes.
Found 46141 validated image filenames belonging to 3260 classes.
Found 46141 validated image filenames belonging to 3260 classes.
(32, 224, 224, 3)


In [48]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Layer
from keras import regularizers
from tensorflow import keras
#from transformers import BeitImageProcessor, BeitForImageClassification
#from keras_cv_attention_models import beit
from transformers import AutoImageProcessor, TFViTModel
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.4),
    tf.keras.layers.experimental.preprocessing.RandomTranslation(height_factor=0.2, width_factor=0),
    tf.keras.layers.experimental.preprocessing.RandomTranslation(height_factor=0, width_factor=0.2),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.2, 0.2),
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.experimental.preprocessing.RandomContrast(0.2),
])
class ExtractPoolerOutput(Layer):
    def call(self, inputs):
        return inputs.pooler_output
#base_model = tf.keras.applications.MobileNetV2(input_shape=(96, 96, 3), include_top=False, weights='imagenet')
base_model = TFViTModel.from_pretrained("google/vit-base-patch16-224-in21k")
image_processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")
#base_model = hub.KerasLayer("https://www.kaggle.com/models/google/efficientnet-v2/frameworks/TensorFlow2/variations/imagenet21k-b0-classification/versions/1")
base_model.trainable = False
model = Sequential()

model.add(data_augmentation)
model.add(base_model)
model.add(ExtractPoolerOutput())
model.add(tf.keras.layers.Dense(3260))
#model.build((32, 224, 224, 3))
#model.summary(show_trainable=True)
#model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalFocalCrossentropy(), metrics=['accuracy'])
#print(len(base_model.layers))

All PyTorch model weights were used when initializing TFViTModel.

All the weights of TFViTModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFViTModel for predictions without further training.


In [50]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt
checkpoint = ModelCheckpoint("best_model.keras", save_best_only=True, save_weights_only=True, monitor='val_loss', mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=20, verbose=1, mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6 , mode='min', verbose=1)
decay_steps = 1000.0
initial_learning_rate = 0.0
warmup_steps = 1000.0
target_learning_rate = 0.1
lr_warmup_decayed_fn = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate, decay_steps, warmup_target=target_learning_rate,
    warmup_steps=warmup_steps
)
sgd_optimizer = tf.keras.optimizers.SGD(learning_rate=0.05, momentum=0.90)
#model.summary(show_trainable=True)
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0), metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=5, name='top_5_categorical_accuracy'), tf.keras.metrics.TopKCategoricalAccuracy(k=3, name='top_3_categorical_accuracy')])
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=25, 
    verbose=1,
    steps_per_epoch=2000,
    validation_steps=len(validation_generator)/32,
    callbacks=[checkpoint, early_stopping, reduce_lr])  

tra_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
plt.figure(figsize=(15,10))
plt.subplot(2,2,1)
plt.plot(tra_acc, label = "Training Accuracy")
plt.plot(val_acc, label = "Validation Accuracy")
plt.legend()
plt.title("Training vs Validation Acc")

tra_loss = history.history['loss']
val_loss = history.history['val_loss']
plt.subplot(2,2,2)
plt.plot(tra_loss, label = "Training Loss")
plt.plot(val_loss, label = "Validation Loss")
plt.legend()
plt.title("Training Loss vs Validation Loss")
plt.show()


Epoch 1/25


2024-02-25 14:44:59.762161: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at conv_ops_impl.h:668 : INVALID_ARGUMENT: input depth must be evenly divisible by filter depth: 224 vs 3


InvalidArgumentError: Graph execution error:

Detected at node sequential_36/tf_vi_t_model_17/vit/embeddings/patch_embeddings/projection/Conv2D defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/leogu/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/home/leogu/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/leogu/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/leogu/.local/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/home/leogu/.local/lib/p

In [ ]:
base_model.trainable = True
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=100, 
    verbose=1,
    initial_epoch = 0,
    steps_per_epoch=500,
    validation_steps=len(validation_generator)/32,
    callbacks=[checkpoint, early_stopping, reduce_lr])  

tra_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
plt.figure(figsize=(15,10))
plt.subplot(2,2,1)
plt.plot(tra_acc, label = "Training Accuracy")
plt.plot(val_acc, label = "Validation Accuracy")
plt.legend()
plt.title("Training vs Validation Acc")

tra_loss = history.history['loss']
val_loss = history.history['val_loss']
plt.subplot(2,2,2)
plt.plot(tra_loss, label = "Training Loss")
plt.plot(val_loss, label = "Validation Loss")
plt.legend()
plt.title("Training Loss vs Validation Loss")
plt.show()